## Install konlpy and Import libraries

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 38.5MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [19]:
import torch
from torchtext import data
import pandas as pd
from konlpy.tag import Komoran 

## Change xlsx file to csv file

In [8]:
dataset = pd.read_excel('/content/drive/MyDrive/korean_sementic_dataset.xlsx')
dataset.to_csv('/content/drive/MyDrive/korean_smentic_dataset_ver_one.csv')

In [20]:
# for compare to csv file 
print(dataset.head(5))

dataset2 = pd.read_csv('/content/drive/MyDrive/korean_smentic_dataset_ver_one.csv')
dataset2.drop(dataset2.columns[0], axis = 1, inplace = True) # drop unknown column
print(dataset2.head(5))
dataset2.to_csv('/content/drive/MyDrive/korean_smentic_dataset_ver_one.csv')

                   Sentence Emotion
0  언니 동생으로 부르는게 맞는 일인가요..??      공포
1              그냥 내 느낌일뿐겠지?      공포
2            아직너무초기라서 그런거죠?      공포
3             유치원버스 사고 낫다던데      공포
4               근데 원래이런거맞나요      공포
                   Sentence Emotion
0  언니 동생으로 부르는게 맞는 일인가요..??      공포
1              그냥 내 느낌일뿐겠지?      공포
2            아직너무초기라서 그런거죠?      공포
3             유치원버스 사고 낫다던데      공포
4               근데 원래이런거맞나요      공포


## Define data_loader Class

In [23]:
komoran = Komoran()

class DataLoader(object):
  def __init__(self, train_fn, batch_size = 64, valid_ratio = .2, device = -1, max_vocab = 99999, min_freq = 5, use_eos = False, shuffle = True):
    # train_fn : train dataset path, max_vocab : max vocab size, min_freq : minimum frequency for loaded word
    super().__init__()

    self.label = data.Field(
        sequential = False,
        use_vocab = True,
        unk_token = False
    )
    self.text = data.Field(
        use_vocab = True,
        batch_first = True,
        include_lengths = False,
        tokenize = komoran # use komoran
    )

    train, valid = data.TabularDataset(
        path = '/content/drive/MyDrive/korean_smentic_dataset_ver_one.csv',
        format = 'csv',
        fields = [
                  ('text', self.text),
                  ('label', self.label),
        ],
    ).split(split_ratio = (1-valid_ratio))

    self.train_loader, self.test_loader = data.BucketIterator.splits(
        (train, valid),
        batch_size = batch_size,
        device = 'cpu',
        shuffle = shuffle,
        sort_key = lambda x: len(x.text),
        sort_within_batch = True, 
    )
    
    self.label.build_vocab(train)
    self.text.build_vocab(train, max_size = max_vocab, min_freq = min_freq)